# TRANSFORMER - TRAINING FROM SCRATCH - TRAIN TỪ ĐẦU

---

In [1]:
!pip install --quiet datasets
!pip install --quiet evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
# Kết nối Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Dataset hay gặp lỗi ** nên cần update lại,restart lại phiên

In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

---

In [3]:
import os, math, copy, torch, re
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tokenizers import ByteLevelBPETokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import PreTrainedTokenizerFast

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#Tạo thư mục lưu token, mô hình
ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)


# dùng dataset y tế mẫu - 112k mẫu
ds = load_dataset("RafaelMPereira/HealthCareMagic-100k-Chat-Format-en")
raw_train = ds["train"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

HealthCareMagic-100k-en.jsonl:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [6]:
#hàm xử lý đầu vào dts, vì chỉ có 1 cột chung 2 loại, nên cần tách riêng
def preprocess_dataset(dataset, max_samples=None):
    ins, outs = [], []
    for i, ex in enumerate(dataset):
        text = ex["text"]
        if "<human>:" in text and "<bot>:" in text:
            before_bot,_,bot_part=text.partition("<bot>:")
            _, _, human_part=before_bot.partition("<human>:")
            q=human_part.strip()
            a=re.sub(r"(?i)LƯU Ý.*$", "", bot_part).strip()
            ins.append(q)
            outs.append(a)
        if max_samples and len(ins) >= max_samples:
            break
    return ins, outs


#lấy tất cả mẫu
full_ins, full_outs = preprocess_dataset(raw_train, max_samples=None)

In [7]:
#Chia 80/10/10 thành train/val/test
# 10% test
train_ins,test_ins, train_outs, test_outs=train_test_split(
    full_ins,
    full_outs,
    test_size=0.1,
    random_state=42
)

#từ train còn lại chia 10% thành val
train_ins,val_ins, train_outs, val_outs=train_test_split(
    train_ins,
    train_outs,
    test_size=0.1,
    random_state=42
)

In [8]:
#BPE tokenizer
tok_dir=os.path.join(ARTIFACT_DIR, "bpe_tok")
if not os.path.isdir(tok_dir):
    bpe=ByteLevelBPETokenizer()
    bpe.train_from_iterator(
        train_ins + train_outs,
        vocab_size=30_000,
        min_frequency=2,special_tokens=["<pad>","<s>","</s>","<unk>"]
    )
    tok=PreTrainedTokenizerFast(tokenizer_object=bpe)
    tok.add_special_tokens({
        "pad_token":"<pad>",
        "bos_token":"<s>",
        "eos_token":"</s>", "unk_token":"<unk>"
    })
    tok.save_pretrained(tok_dir)
else:
    tok=PreTrainedTokenizerFast.from_pretrained(tok_dir)

vocab_size= tok.vocab_size
pad_id=tok.pad_token_id
bos_id= tok.bos_token_id
eos_id=tok.eos_token_id

# Lớp Dataset

In [9]:
# Lớp ataset + DataLoader
class MedDialogDataset(Dataset):
    def __init__(self,ins,outs,tokenizer,max_len=128):
        self.ins,self.outs=ins,outs
        self.tok,self.max_len= tokenizer,max_len

    def __len__(self): return len(self.ins)
    def encode(self, txt):
        ids= self.tok.encode(txt, add_special_tokens=False)
        if len(ids)>self.max_len-2:
          ids=ids[:self.max_len-2]
        return [bos_id]+ids+[eos_id]

    def __getitem__(self,i):
        src= torch.tensor(self.encode("Patient: "+self.ins[i]), dtype=torch.long)
        tgt=torch.tensor(self.encode("Doctor: "+self.outs[i]),dtype=torch.long)
        return src,tgt

def collate_fn(batch):
    pad=pad_id
    srcs,tgts=zip(*batch)
    srcs=pad_sequence(srcs, batch_first=True, padding_value=pad)
    tgts=pad_sequence(tgts, batch_first=True, padding_value=pad)
    # masks 1. # (B,S_src), pad -> # (B,1,S_tgt), tgt_mask -> # (B,1,L,L)
    src_kpm= srcs.eq(pad)
    pad_mask= tgts.ne(pad).unsqueeze(1)
    L= tgts.size(1)
    causal= torch.triu(torch.ones((1,L,L),dtype=torch.bool),1)==0
    tgt_mask= pad_mask & causal
    return srcs.to(device), tgts.to(device), src_kpm.to(device), tgt_mask.to(device)

#batch_size chung là 32
bs=32

train_loader= DataLoader(MedDialogDataset(train_ins,train_outs,tok),batch_size=bs,shuffle=True, collate_fn=collate_fn)
val_loader=DataLoader(MedDialogDataset(val_ins,  val_outs,  tok),batch_size=bs,shuffle=False,collate_fn=collate_fn)
test_loader= DataLoader(MedDialogDataset(test_ins, test_outs, tok),batch_size=bs,shuffle=False,collate_fn=collate_fn)

# Transformer

In [10]:
#Scratch Transformer (Pre-Norm + Weight-tie)
class Embedder(nn.Module):
    def __init__(self, V, d, pad_idx):
        super().__init__()
        self.e=nn.Embedding(V,d, padding_idx=pad_idx)
        self.d= d

    def forward(self,x):
      return self.e(x)*math.sqrt(self.d)

class PositionalEncoder(nn.Module):
    def __init__(self, d, drop=0.1, max_len=2000):
        super().__init__()
        pe= torch.zeros(max_len, d)
        pos= torch.arange(max_len).unsqueeze(1)
        div=torch.exp(torch.arange(0,d,2)*(-math.log(10000)/d))
        pe[:,0::2]=torch.sin(pos*div)
        pe[:,1::2]=torch.cos(pos*div)
        self.register_buffer("pe", pe.unsqueeze(0))
        self.drop = nn.Dropout(drop)

    def forward(self,x):
        return self.drop(x+self.pe[:,:x.size(1)])

class MHA_Block(nn.Module):
    def __init__(self,d,h,drop=0.1):
        super().__init__()
        self.att=nn.MultiheadAttention(d,h,dropout=drop,batch_first=True)
        self.drop= nn.Dropout(drop)

    def forward(self,q,k,v,mask,kpm):
        # mask -> (L,L)
        # kpm ->  key padding mask (B,S)
        y,_ = self.att(q,k,v,
                      attn_mask=mask,
                      key_padding_mask=kpm)
        return self.drop(y)

class FFN(nn.Module):
    def __init__(self,d,dff,drop=0.1):
        super().__init__()
        self.net=nn.Sequential(
          nn.Linear(d,dff),
          nn.ReLU(),
          nn.Dropout(drop),
          nn.Linear(dff,d),
          nn.Dropout(drop)
        )
    def forward(self,x):
      return self.net(x)

class EncoderLayer(nn.Module):
    def __init__(self,d,h,dff,drop):
        super().__init__()
        self.ln1=nn.LayerNorm(d)
        self.ln2=nn.LayerNorm(d)
        self.mha=MHA_Block(d,h,drop)
        self.ff=FFN(d,dff,drop)

    def forward(self,x,src_kpm):
        x2=self.ln1(x)
        x=x+ self.mha(x2,x2,x2,mask=None, kpm=src_kpm)
        x2=self.ln2(x)
        return x+ self.ff(x2)

class DecoderLayer(nn.Module):
    def __init__(self,d,h,dff,drop):
        super().__init__()
        self.ln1=nn.LayerNorm(d)
        self.ln2=nn.LayerNorm(d)
        self.ln3=nn.LayerNorm(d)
        #3 sub-layer
        self.self_att=MHA_Block(d,h,drop)
        self.cross_att= MHA_Block(d,h,drop)
        #fnn
        self.ff=FFN(d,dff,drop)

    def forward(self,x,mem,src_kpm,tgt_kpm,causal_mask):
        x2 = self.ln1(x)
        x  = x + self.self_att(x2,x2,x2, mask=causal_mask, kpm=tgt_kpm)
        x2 = self.ln2(x)
        x  = x + self.cross_att(x2,mem,mem, mask=None, kpm=src_kpm)
        x2 = self.ln3(x)
        return x + self.ff(x2)

class Transformer(nn.Module):
    def __init__(self,V,d=512,h=8,enc_layers=6,dec_layers=6,dff=2048,drop=0.1,pad_idx=0):
        super().__init__()
        self.emb=Embedder(V,d,pad_idx)
        self.pos= PositionalEncoder(d,drop)
        self.encs=nn.ModuleList([EncoderLayer(d,h,dff,drop) for _ in range(enc_layers)])
        self.decs= nn.ModuleList([DecoderLayer(d,h,dff,drop) for _ in range(dec_layers)])
        self.ln=nn.LayerNorm(d)
        self.outp=nn.Linear(d,V, bias=False)
        #weight-tying
        self.outp.weight=self.emb.e.weight

    def forward(self,src,tgt,src_kpm,tgt_kpm,causal_mask):
        #src -> (B,S_src),tgt -> (B,S_tgt)
        mem=self.pos(self.emb(src))
        for l in self.encs:
            mem=l(mem, src_kpm)
        x = self.pos(self.emb(tgt))
        for l in self.decs:
            x=l(x, mem, src_kpm, tgt_kpm, causal_mask)
            # -> (B,S_tgt,V)
        return self.outp(self.ln(x))

In [11]:
#Loss & Scheduler và Early stop
class LabelSmoothingLoss(nn.Module):
    def __init__(self,C,s=0.1):
        super().__init__()
        self.conf=1-s
        self.s=s
        self.C=C

    def forward(self,pred,tgt):
        #pred -> (N,V), tgt -> (N,)
        with torch.no_grad():
            true = torch.full_like(pred, self.s/(self.C-1))
            true.scatter_(1, tgt.unsqueeze(1), self.conf)

        return F.kl_div(F.log_softmax(pred,dim=1), true, reduction="batchmean")

class NoamOpt:
    def __init__(self,d_model,fac,warm,opt):
        self.optimizer=opt
        self.n=0
        self.w=warm
        self.f=fac
        self.d=d_model

    def step(self):
        self.n+=1
        lr = self.f*(self.d**-0.5)*min(self.n**-0.5, self.n*(self.w**-1.5))
        for g in self.optimizer.param_groups:
          g["lr"]=lr
        self.optimizer.step()

    def zero_grad(self):
      self.optimizer.zero_grad()

model= Transformer(vocab_size,pad_idx=pad_id).to(device)
crit= LabelSmoothingLoss(vocab_size,s=0.1)
base_opt= torch.optim.Adam(model.parameters(),betas=(0.9,0.98),eps=1e-9)
opt=NoamOpt(d_model=512,fac=1,warm=4000,opt=base_opt)

best_val,wait, patience = float('inf'), 0, 3
num_epochs= 10

#Masks helpers
def make_padding_mask(seq):
  # -> (B,S)
  return seq.eq(pad_id)


def make_causal_mask(L):
    return torch.triu(torch.full((L,L), float('-inf')),1).to(device)

# Train

In [ ]:
import json

In [11]:
#TRAIN
#LƯU lại kq
metrics= []

best_val= float("inf")
wait, patience= 0, 3

for ep in range(1, num_epochs+1):
    #train
    model.train()
    total_train = 0.0
    for src, tgt, src_kpm, tgt_mask in train_loader:
        dec_in=tgt[:, :-1]
        dec_tgt=tgt[:, 1:]
        src_kpm_idx= src_kpm
        tgt_kpm_idx= dec_in.eq(pad_id)
        causal_m= make_causal_mask(dec_in.size(1))

        opt.zero_grad()
        out=model(src, dec_in, src_kpm_idx, tgt_kpm_idx, causal_m)
        loss= crit(out.reshape(-1, vocab_size),dec_tgt.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        total_train+=loss.item()
    avg_tr=total_train/len(train_loader)

    #validation
    model.eval()
    total_val= 0.0
    with torch.no_grad():
        for src,tgt,src_kpm,tgt_mask in val_loader:
            dec_in= tgt[:, :-1]
            dec_tgt= tgt[:, 1:]
            tgt_kpm_idx = dec_in.eq(pad_id)
            causal_m=make_causal_mask(dec_in.size(1))
            out = model(src, dec_in, src_kpm, tgt_kpm_idx, causal_m)
            total_val+=crit(out.reshape(-1, vocab_size),dec_tgt.reshape(-1)).item()
    avg_val=total_val/len(val_loader)

    print(f"Epoch {ep} | Train Loss={avg_tr:.4f} | Val Loss={avg_val:.4f}")

    #lưu kq từng epoch
    metrics.append({"epoch":ep,"train_loss":avg_tr,"val_loss":avg_val})

    #early stopping và checkpoint (lưu lại mô hình mỗi epoch để lỡ bị dừng còn có cái mà dùng)
    if avg_val<best_val:
        best_val,wait=avg_val, 0
        torch.save(model.state_dict(), os.path.join(ARTIFACT_DIR, "best_model.pt"))
        print(" Đã lưu mô hình tốt nhất")
    else:
        wait+=1
        if wait>=patience:
            print(f"Early stopping kích hoạt)")
            break

metrics_path = os.path.join(ARTIFACT_DIR, "metrics.json")
with open(metrics_path, "w") as fp:
    json.dump(metrics, fp, indent=2)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(



---

# Đánh giá Test

In [12]:
!pip install evaluate

In [13]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e5500349da8945b926b818dc0fcb2731e9728b5038b2f145479af3517d9bc2ac
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [14]:
ckpt = os.path.join(ARTIFACT_DIR, "best_model.pt")
model.load_state_dict(torch.load(ckpt, map_location=device))
model.eval()

Transformer(
  (emb): Embedder(
    (e): Embedding(30000, 512, padding_idx=0)
  )
  (pos): PositionalEncoder(
    (drop): Dropout(p=0.1, inplace=False)
  )
  (encs): ModuleList(
    (0-5): 6 x EncoderLayer(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mha): MHA_Block(
        (att): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (drop): Dropout(p=0.1, inplace=False)
      )
      (ff): FFN(
        (net): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=2048, out_features=512, bias=True)
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (decs): ModuleList(
    (0-5): 6 x DecoderLayer(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=Tr

In [ ]:
@torch.no_grad()
def scratch_generate(question, max_new=128):
    prompt = f"Patient: {question}\nDoctor:"
    ids    = tok.encode(prompt, add_special_tokens=False)
    seq    = torch.tensor([ [bos_id] + ids ], device=device)
    for _ in range(max_new):
        L = seq.size(1)
        # causal mask
        causal = torch.triu(torch.full((L, L), float('-inf'),
                                       device=device), 1)
        out = model(seq, seq, seq.eq(pad_id), seq.eq(pad_id), causal)
        nxt = out[0, -1].argmax().item()
        seq = torch.cat([seq, torch.tensor([[nxt]], device=device)], dim=1)
        if nxt == eos_id:
            break
    # decode bỏ BOS và skip_special_tokens
    toks = seq.squeeze().tolist()[1:]
    dec  = tok.decode(toks, skip_special_tokens=True)
    return dec.split("Doctor:")[-1].strip()

In [ ]:
ce, nll, n_tok = torch.nn.CrossEntropyLoss(ignore_index=pad_id, reduction="sum"), 0.0, 0
model.eval()
with torch.no_grad():
    for src, tgt, src_kpm, tgt_mask in test_loader:
        dec_in  = tgt[:, :-1]
        dec_tgt = tgt[:, 1:]
        L = dec_in.size(1)
        causal = torch.triu(torch.full((L,L), float('-inf'), device=device), 1)
        logits = model(src, dec_in, src_kpm, dec_in.eq(pad_id), causal)
        loss   = ce(logits.reshape(-1, vocab_size), dec_tgt.reshape(-1))
        valid  = (dec_tgt != pad_id).sum().item()
        nll   += loss.item()
        n_tok += valid

log_ppl = nll / n_tok
ppl     = math.exp(log_ppl)

In [ ]:
bleu  = evaluate.load("bleu")
rouge = evaluate.load("rouge")

hyps, refs = [], []
for q, a in zip(test_ins[:len(test_loader.dataset)], test_outs[:len(test_loader.dataset)]):
    hyps.append(scratch_generate(q))
    refs.append([a])

In [ ]:
bleu_score  = bleu.compute(predictions=hyps, references=refs)["bleu"] * 100
rouge_score = rouge.compute(
    predictions=hyps,
    references=[r[0] for r in refs]
)["rougeL"] * 100

print(f"Log‐PPL: {log_ppl:.4f} | PPL: {ppl:.2f}")
print(f"BLEU-4 : {bleu_score:.2f}")
print(f"ROUGE-L: {rouge_score:.2f}")

Log‐PPL: 4.8026 | PPL: 121.83
BLEU-4 : 0.14
ROUGE-L: 5.72


=> KQ ko ổn cho thấy nó chỉ đang tạo sinh 1 cách random

=> nguyên nhân lỡ nhét luôn cả prompt và output vào cùng một chu trình mà không tách encoder/decoder rõ ràng, dẫn đến generation không có ý nghĩa.

In [ ]:
import datetime, textwrap, random

In [ ]:
#Xuất 25 mẫu cho human evaluation
rand_idx = random.sample(range(test_loader), 25)
out_path = os.path.join(ARTIFACT_DIR, f"human_eval_scratch_{datetime.date.today()}.txt")
with open(out_path, "w", encoding="utf-8") as fp:
    for k, idx in enumerate(rand_idx, 1):
        q, ref = test_ins[idx], test_outs[idx]
        pred    = scratch_generate(q)
        fp.write("="*80 + "\n")
        fp.write(f"[{k:02d}] QUESTION:\n{textwrap.fill(q,70)}\n\n")
        fp.write(f"REFERENCE:\n{textwrap.fill(ref,70)}\n\n")
        fp.write(f"MODEL:\n{textwrap.fill(pred,70)}\n\n")
print(f"✓ Đã xuất 25 mẫu human‐eval lên {out_path}")

# khác

=> GIÚP TUÂN THỦ autoregressive decoding (“từ trái sang phải”):

In [23]:
import torch
import torch.nn.functional as F

@torch.no_grad()
def scratch_generate(question,
                     max_len=128,
                     do_sample=True,
                     top_k=50,
                     top_p=0.9,
                     temperature=1.0,
                     device=device):
    model.eval()
    # --- encode nguồn ---
    src_ids = tok.encode("Patient: " + question,
                         add_special_tokens=False)[:max_len-2]
    src = torch.tensor([[bos_id] + src_ids + [eos_id]],
                       device=device)
    src_kpm = src.eq(pad_id)

    # --- encoder (lưu memory) ---
    mem = model.pos(model.emb(src))
    for layer in model.encs:
        mem = layer(mem, src_kpm)

    # --- decoder sampling/greedy ---
    out_ids = [bos_id]
    for _ in range(max_len):
        dec_in = torch.tensor([out_ids], device=device)
        tgt_kpm = dec_in.eq(pad_id)
        causal = make_causal_mask(dec_in.size(1))

        # raw logits for next token
        logits = model(src, dec_in, src_kpm, tgt_kpm, causal)[0, -1]

        # apply temperature
        logits = logits / max(temperature, 1e-8)

        if do_sample:
            # top-k: loại bỏ các token ngoài  top_k xác suất cao nhất
            if top_k > 0:
                topk_vals, topk_idx = torch.topk(logits, top_k)
                min_topk = topk_vals[-1]
                logits[logits < min_topk] = -float('Inf')

            # top-p (nucleus): giữ smallest set token có tổng prob ≥ top_p
            if top_p < 1.0:
                sorted_logits, sorted_idx = torch.sort(logits, descending=True)
                sorted_probs = F.softmax(sorted_logits, dim=-1)
                cum_probs = torch.cumsum(sorted_probs, dim=-1)
                # mask tokens beyond cumulative threshold
                mask = cum_probs > top_p
                # ensure at least one token
                mask[..., 1:] = mask[..., :-1].clone()
                mask[..., 0] = False
                remove_idx = sorted_idx[mask]
                logits[remove_idx] = -float('Inf')

            # sample next token
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1).item()
        else:
            # greedy
            next_id = int(torch.argmax(logits))

        out_ids.append(next_id)
        if next_id == eos_id:
            break

    # decode và strip special tokens + prefix
    gen = tok.decode(out_ids[1:], skip_special_tokens=True)
    return re.sub(r"^Doctor:\s*", "", gen).strip()


In [ ]:
import math
import torch
import evaluate

In [ ]:
ce = torch.nn.CrossEntropyLoss(ignore_index=pad_id, reduction="sum")
nll, n_tok = 0.0, 0

model.eval()
with torch.no_grad():
    for src, tgt, src_kpm, tgt_mask in test_loader:
        dec_in  = tgt[:, :-1]
        dec_tgt = tgt[:, 1:]
        L = dec_in.size(1)
        causal = torch.triu(torch.full((L, L), float('-inf'), device=device), 1)
        logits = model(src, dec_in, src_kpm, dec_in.eq(pad_id), causal)
        loss   = ce(logits.reshape(-1, vocab_size), dec_tgt.reshape(-1))
        valid  = (dec_tgt != pad_id).sum().item()
        nll   += loss.item()
        n_tok += valid

log_ppl = nll / n_tok
ppl     = math.exp(log_ppl)

bleu  = evaluate.load("bleu")
rouge = evaluate.load("rouge")

hyps = []
refs = []
for q, a in zip(test_ins, test_outs):
  # thêm sampling (top-k, top-p, temperature)
    hyps.append(scratch_generate(
        q,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7
    ))
    refs.append([a])

bleu_score  = bleu.compute(predictions=hyps, references=refs)["bleu"] * 100
rouge_score = rouge.compute(predictions=hyps, references=[r[0] for r in refs])["rougeL"] * 100

In [1]:
print(f"Log‐PPL: {log_ppl:.4f} | PPL: {ppl:.2f}")
print(f"BLEU-4 : {bleu_score:.2f}")
print(f"ROUGE-L: {rouge_score:.2f}")

Log‐PPL: 6.6922 | PPL: 230.4
BLEU-4 : 2.31
ROUGE-L: 14.91


In [25]:
import random, datetime, textwrap

In [26]:
rand_idx = random.sample(range(len(test_ins)), 25)

out_path = os.path.join(
    ARTIFACT_DIR,
    f"human_eval_scratch_{datetime.date.today()}.txt"
)

with open(out_path, "w", encoding="utf-8") as fp:
    for k, idx in enumerate(rand_idx, 1):
        q   = test_ins[idx]
        ref = test_outs[idx]
        pred = scratch_generate(q)

        fp.write("="*80 + "\n")
        fp.write(f"[{k:02d}] QUESTION:\n{textwrap.fill(q,70)}\n\n")
        fp.write(f"REFERENCE:\n{textwrap.fill(ref,70)}\n\n")
        fp.write(f"MODEL:\n{textwrap.fill(pred,70)}\n\n")

print(f"Đã xuất 25 mẫu human‐eval lên {out_path}")

Đã xuất 25 mẫu human‐eval lên artifacts/human_eval_scratch_2025-05-21.txt
